In [50]:
import torch
#import matplotlib.pyplot as plt
from model import *
from others.helpers import *

%reload_ext autoreload
%autoreload 2

Import images, sample and transform into float (to correspond to model)

In [51]:
noisy_imgs_1, noisy_imgs_2 = torch.load('../data/train_data.pkl')
noisy_imgs_1, noisy_imgs_2 = noisy_imgs_1.float()/255, noisy_imgs_2.float()/255

img_samples_1, img_samples_2 = noisy_imgs_1[:100], noisy_imgs_2[:100]
print(img_samples_1.shape, img_samples_2.shape)

torch.Size([100, 3, 32, 32]) torch.Size([100, 3, 32, 32])


In [ ]:
import matplotlib.pyplot as plt
idx = 60
plt.imshow(img_samples_1[idx,:,:,:].permute(1,2,0))
plt.show()
plt.imshow(img_samples_2[idx,:,:,:].permute(1,2,0))
plt.show()

Parameters

In [52]:
# architecture
in_channels = out_channels = noisy_imgs_1.shape[1]
conv_by_level = 2
features = [16,32,64]
pooling_type = 'max'
batch_norm = True
dropout = 0
model_ARGS = [in_channels, out_channels, conv_by_level, features, pooling_type, batch_norm, dropout]

# training
optimizer = 'SGD' #torch.optim.SGD(Model.parameters(), lr=0.01, momentum = 0.9) 
loss_func = nn.MSELoss() #psnr loss provided in the pdf ? It's in the others/helpers.py # nn.lL1Loss()
batch_size = 10
num_epoch = 5
train_ARGS = [optimizer, loss_func, batch_size, num_epoch]

# control arg
shape_control = True

Model test

In [53]:
Noise2noise = Model()
Noise2noise = Model(model_ARGS, train_ARGS, shape_control)
Noise2noise(img_samples_1)
print('Forward pass done.')

Original shape: torch.Size([100, 3, 32, 32])
Level 1: conv shape: torch.Size([100, 16, 32, 32]), pooled shape: torch.Size([100, 16, 16, 16])
Level 2: conv shape: torch.Size([100, 32, 16, 16]), pooled shape: torch.Size([100, 32, 8, 8])
Level 3: conv shape: torch.Size([100, 64, 8, 8]), pooled shape: torch.Size([100, 64, 4, 4])
Deepest shape: torch.Size([100, 128, 4, 4])
Level 3: upscaled shape: torch.Size([100, 64, 8, 8]), concat shape: torch.Size([100, 128, 8, 8]), conv shape: torch.Size([100, 64, 8, 8])
Level 2: upscaled shape: torch.Size([100, 32, 16, 16]), concat shape: torch.Size([100, 64, 16, 16]), conv shape: torch.Size([100, 32, 16, 16])
Level 1: upscaled shape: torch.Size([100, 16, 32, 32]), concat shape: torch.Size([100, 32, 32, 32]), conv shape: torch.Size([100, 16, 32, 32])
Final shape: torch.Size([100, 3, 32, 32])
Forward pass done.


In [54]:
Noise2noise = Model(model_ARGS, train_ARGS, shape_control=False)

In [55]:
Noise2noise.train(img_samples_1, img_samples_2)

Batch 5 : 0.18 sec
Epoch [1/5], Train loss: 0.2355
Epoch [1/5], Train loss: 0.2355 Validation loss: 0.0951
=> Saving checkpoint
Batch 5 : 0.17 sec
Epoch [2/5], Train loss: 0.0998
Epoch [2/5], Train loss: 0.0998 Validation loss: 0.1110
Batch 5 : 0.25 sec
Epoch [3/5], Train loss: 0.0511
Epoch [3/5], Train loss: 0.0511 Validation loss: 0.0501
=> Saving checkpoint
Batch 5 : 0.18 sec
Epoch [4/5], Train loss: 0.0375
Epoch [4/5], Train loss: 0.0375 Validation loss: 0.0725
Batch 5 : 0.21 sec
Epoch [5/5], Train loss: 0.0308
Epoch [5/5], Train loss: 0.0308 Validation loss: 0.0597
Training finished with best best results at epoch 3 | Validation loss : 0.0501 | Training loss : 0.0511


In [56]:
Noise2noise.load_pretrained_model()

=> Loading checkpoint from a trained model at the best epoch 3


In [60]:
noisy_test , clean_test = torch.load ('../data/val_data.pkl')
noisy_test, clean_test = noisy_test.float()/255, clean_test.float()/255
noisy_test_sample,  clean_test_sample = noisy_test[:100], clean_test[:100]

In [ ]:
import matplotlib.pyplot as plt
idx = 90
plt.imshow(noisy_test_sample[idx,:,:,:].permute(1,2,0))
plt.show()
plt.imshow(clean_test_sample[idx,:,:,:].permute(1,2,0))
plt.show()

In [ ]:
test_output = Noise2noise.predict(noisy_test_sample)

In [ ]:
print(psnr(test_output, clean_test_sample))